<a href="https://colab.research.google.com/github/mchatiwat/ClassDemoFirebaseDB/blob/master/Alpaca_Dynamic_Mean_Reversion_%26_Momentum_Breakout_Strategy_Paper_trading_worked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries in Google Colab or your environment
# !pip install alpaca-py
# !pip install alpaca-trade-api pandas pandas_ta numpy==1.26.4
#============================================================
# !pip install numpy==1.26.4 # Downgrade numpy if needed for pandas_ta compatibility

# Import necessary components from the correct libraries
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, QueryOrderStatus
from alpaca.trading.enums import OrderSide, TimeInForce, OrderStatus
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data import TimeFrame

import pandas as pd
import pandas_ta as ta # For calculating technical indicators easily
import numpy as np
import time
import os
from datetime import datetime, timedelta, timezone # <-- Import timezone

# --- Configuration ---
# Replace with your Alpaca Paper Trading API keys
# It's best practice to use environment variables for keys
API_KEY = "PKJ1OWR6GEGILCT2LLFB"  # Replace with your key
API_SECRET = "DyogjSqfBiLX8pT7G3OscRQcv2iiOysanEb51EFC" # Replace with your secret
# BASE_URL is not directly used by the newer clients but kept for potential REST usage
BASE_URL = "https://paper-api.alpaca.markets/v2"

# List of stocks to trade
STOCKS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'LLV', 'JPM', 'V', 'WMT', 'NFLX',
          'XOM', 'PG', 'UNH', 'HD', 'ABBV', 'JNJ', 'PLTR', 'MA', 'BAC', 'BRK.B', 'VRT'] # Ensure GOOGL existed throughout the period


# Strategy Parameters
SMA_SHORT = 10 # Short-term Simple Moving Average period
SMA_LONG = 30 # Long-term Simple Moving Average period (for context, not direct signal)
BBANDS_PERIOD = 20 # Bollinger Bands period
BBANDS_STD_DEV = 2 # Bollinger Bands standard deviation
RSI_PERIOD = 10 # RSI period
RSI_OVERBOUGHT = 65 # RSI overbought threshold
RSI_OVERSOLD = 35 # RSI oversold threshold
ROC_PERIOD = 10 # Rate of Change period
# *** Use BBB (Bandwidth Percentage) Threshold instead of BBW (Width) ***
BBB_THRESHOLD_PERCENTILE = 50 # Bollinger Band Width PERCENTAGE percentile to switch regime
LOOKBACK_DAYS_FOR_BBB = 33 # Days of historical data to calculate BBB percentile
CAPITAL_PER_STOCK_PERCENT = 0.90 # Allocate 10% of equity to each stock max

# --- Alpaca API Clients ---
try:
    # Use paper=True for paper trading
    trading_client = TradingClient(API_KEY, API_SECRET, paper=True)
    # No need to specify paper for data client
    data_client = StockHistoricalDataClient(API_KEY, API_SECRET)
except Exception as e:
    print(f"Error initializing Alpaca clients: {e}")
    # Exit or handle error appropriately
    exit()

# --- Helper Functions ---

def get_historical_data(symbols, timeframe, start_date, end_date):
    """Fetches historical bar data for multiple symbols and handles MultiIndex timezone."""
    try:
        request_params = StockBarsRequest(
            symbol_or_symbols=symbols,
            timeframe=timeframe, # Pass the TimeFrame enum directly
            start=start_date, # Expects datetime objects
            end=end_date,     # Expects datetime objects
            adjustment='raw' # or 'split', 'dividend', 'all'
        )
        print(f"Requesting data from {start_date} to {end_date}") # Debug print
        bars = data_client.get_stock_bars(request_params)
        # Convert to DataFrame and organize
        df = bars.df

        # Handle MultiIndex Timezone
        if not df.empty and isinstance(df.index, pd.MultiIndex):
            # Alpaca SDK usually returns 'symbol' and 'timestamp' levels
            if 'timestamp' in df.index.names:
                 timestamp_level_name = 'timestamp'
            else:
                 # Fallback assuming timestamp is the last level if name not found
                 timestamp_level_name = df.index.names[-1]
                 print(f"Warning: Timestamp level name not found, assuming '{timestamp_level_name}'.")

            timestamp_level_index = df.index.names.index(timestamp_level_name)

            # Check timezone of the timestamp level
            if df.index.levels[timestamp_level_index].tz is None:
                # Localize the timestamp level to UTC if naive
                df.index = df.index.set_levels(
                    df.index.levels[timestamp_level_index].tz_localize('UTC'),
                    level=timestamp_level_name # Use name instead of index
                )
            else:
                # Convert the timestamp level to UTC if it's already localized but different
                df.index = df.index.set_levels(
                    df.index.levels[timestamp_level_index].tz_convert('UTC'),
                    level=timestamp_level_name # Use name instead of index
                )
        elif not df.empty: # Handle single index DataFrame
             if df.index.tz is None:
                df = df.tz_localize('UTC')
             else:
                df = df.tz_convert('UTC') # Ensure it's UTC if already localized

        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return pd.DataFrame() # Return empty DataFrame on error

def calculate_indicators(df):
    """Calculates technical indicators using pandas_ta. Handles MultiIndex."""
    if df.empty:
        print("calculate_indicators: Input DataFrame is empty.")
        return df
    try:
        # Make a copy to avoid modifying the original DataFrame slice
        df_out = df.copy() # Work on a copy

        # Define indicator column names first for clarity
        sma_short_col = f'SMA_{SMA_SHORT}'
        sma_long_col = f'SMA_{SMA_LONG}'
        bbl_col = f'BBL_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        bbm_col = f'BBM_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        bbu_col = f'BBU_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        # *** Use BBB column name from pandas_ta ***
        bbb_col = f'BBB_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}' # Bandwidth Percentage
        bbp_col = f'BBP_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}' # %B Percent
        rsi_col = f'RSI_{RSI_PERIOD}'
        roc_col = f'ROC_{ROC_PERIOD}'
        bbb_threshold_col = 'bbb_threshold' # Threshold based on BBB

        # Calculate indicators using groupby for MultiIndex DataFrames
        if isinstance(df_out.index, pd.MultiIndex):
            symbol_level = df_out.index.names.index('symbol') # Find the level index for 'symbol'
            print(f"Calculating indicators group-wise for MultiIndex DataFrame...")

            # Use transform for indicators returning a single series - applied directly to df_out
            df_out[sma_short_col] = df_out.groupby(level=symbol_level)['close'].transform(lambda x: ta.sma(x, length=SMA_SHORT))
            df_out[sma_long_col] = df_out.groupby(level=symbol_level)['close'].transform(lambda x: ta.sma(x, length=SMA_LONG))
            df_out[rsi_col] = df_out.groupby(level=symbol_level)['close'].transform(lambda x: ta.rsi(x, length=RSI_PERIOD))
            df_out[roc_col] = df_out.groupby(level=symbol_level)['close'].transform(lambda x: ta.roc(x, length=ROC_PERIOD))

            # Calculate BBands separately for each group and combine
            all_bbands = []
            expected_bb_cols = [bbl_col, bbm_col, bbu_col, bbb_col, bbp_col] # Expect BBB now
            for name, group in df_out.groupby(level=symbol_level):
                bbands_df = ta.bbands(group['close'], length=BBANDS_PERIOD, std=BBANDS_STD_DEV)
                if bbands_df is not None and not bbands_df.empty:
                    all_bbands.append(bbands_df)
                else:
                    print(f"Warning: ta.bbands returned empty for group {name}. Filling with NaNs.")
                    nan_df = pd.DataFrame(np.nan, index=group.index, columns=expected_bb_cols)
                    all_bbands.append(nan_df)

            # Concatenate all the bbands results
            if all_bbands:
                combined_bbands_df = pd.concat(all_bbands)
                df_out = df_out.join(combined_bbands_df, how='left')
                print(f"BBands columns added. Columns now: {df_out.columns.tolist()}")
            else:
                print("Warning: No BBands results were generated.")


            # Calculate BBB threshold group-wise based on the BBB column
            if bbb_col in df_out.columns:
                 def rolling_percentile(x):
                      if not isinstance(x, pd.Series): x = pd.Series(x)
                      x_clean = x.dropna()
                      if x_clean.empty: return pd.Series(np.nan, index=x.index)
                      min_lookback = max(BBANDS_PERIOD, 10)
                      if len(x_clean) >= LOOKBACK_DAYS_FOR_BBB: # Use BBB lookback period
                           window_size = LOOKBACK_DAYS_FOR_BBB
                           if window_size >= min_lookback:
                                return x.rolling(window=window_size, min_periods=min_lookback).apply(lambda y: np.percentile(y.dropna(), BBB_THRESHOLD_PERCENTILE) if not y.dropna().empty else np.nan, raw=False) # Use BBB threshold percentile
                           else: return pd.Series(np.nan, index=x.index)
                      elif len(x_clean) >= min_lookback:
                           return x.expanding(min_periods=min_lookback).apply(lambda y: np.percentile(y.dropna(), BBB_THRESHOLD_PERCENTILE) if not y.dropna().empty else np.nan, raw=False) # Use BBB threshold percentile
                      else: return pd.Series(np.nan, index=x.index)

                 try:
                      # Apply using the BBB column
                      df_out[bbb_threshold_col] = df_out.groupby(level=symbol_level)[bbb_col].transform(rolling_percentile)
                 except Exception as transform_err:
                      print(f"Warning: Groupby transform failed for rolling_percentile on {bbb_col} ({transform_err}), falling back to apply.")
                      try:
                           df_out[bbb_threshold_col] = df_out.groupby(level=symbol_level, group_keys=False)[bbb_col].apply(rolling_percentile)
                      except Exception as apply_err:
                           print(f"Error during groupby apply fallback for rolling_percentile on {bbb_col}: {apply_err}")
                           df_out[bbb_threshold_col] = np.nan
            else:
                 print(f"Warning: BBB column '{bbb_col}' was not found after joining BBands. Cannot calculate threshold.")
                 df_out[bbb_threshold_col] = np.nan


        else: # Single symbol DataFrame
            print(f"Calculating indicators for single symbol DataFrame...")
            df_out.ta.sma(length=SMA_SHORT, append=True)
            df_out.ta.sma(length=SMA_LONG, append=True)
            # Calculate bbands directly and append
            bbands_result = df_out.ta.bbands(length=BBANDS_PERIOD, std=BBANDS_STD_DEV, append=True)
            if bbands_result is None or bbands_result.empty:
                 print(f"Warning: ta.bbands returned empty for single symbol. BBB calculation skipped.")

            df_out.ta.rsi(length=RSI_PERIOD, append=True)
            df_out.ta.roc(length=ROC_PERIOD, append=True)

            # Calculate BBB threshold
            if bbb_col in df_out.columns:
                 if not df_out[bbb_col].isnull().all():
                     min_lookback = max(BBANDS_PERIOD, 10)
                     if len(df_out) >= LOOKBACK_DAYS_FOR_BBB: # Use BBB lookback
                         window_size = LOOKBACK_DAYS_FOR_BBB
                         if window_size >= min_lookback:
                             df_out[bbb_threshold_col] = df_out[bbb_col].rolling(window=window_size, min_periods=min_lookback).apply(lambda x: np.percentile(x.dropna(), BBB_THRESHOLD_PERCENTILE) if not x.dropna().empty else np.nan, raw=False) # Use BBB percentile
                         else: df_out[bbb_threshold_col] = np.nan
                     elif len(df_out) >= min_lookback:
                         df_out[bbb_threshold_col] = df_out[bbb_col].expanding(min_periods=min_lookback).apply(lambda x: np.percentile(x.dropna(), BBB_THRESHOLD_PERCENTILE) if not x.dropna().empty else np.nan, raw=False) # Use BBB percentile
                     else: df_out[bbb_threshold_col] = np.nan
                 else: df_out[bbb_threshold_col] = np.nan
            else:
                 print(f"Warning: BBB column '{bbb_col}' not found for single symbol. Threshold calculation skipped.")
                 df_out[bbb_threshold_col] = np.nan


        # Keep rows with NaNs, checks happen later
        print(f"Indicator calculation complete. DataFrame shape: {df_out.shape}")
        return df_out # Return the modified copy

    except Exception as e:
        print(f"Error calculating indicators: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()


def determine_regime(stock_data):
    """Determines market regime based on Bollinger Band Width PERCENTAGE (BBB)."""
    # Expects data for a SINGLE stock
    # *** Use BBB column name ***
    bbb_col = f'BBB_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
    bbb_threshold_col = 'bbb_threshold' # Use consistent name

    # Check if essential columns exist in the stock_data DataFrame
    if stock_data.empty or bbb_threshold_col not in stock_data.columns or bbb_col not in stock_data.columns:
        symbol = stock_data.index.get_level_values('symbol')[0] if isinstance(stock_data.index, pd.MultiIndex) and not stock_data.empty and 'symbol' in stock_data.index.names else 'stock'
        print(f"Warning: Cannot determine regime for {symbol}, missing required columns ({bbb_col}, {bbb_threshold_col}). Columns present: {stock_data.columns.tolist()}")
        return "UNKNOWN"

    # Get the latest row safely
    try:
        latest_row = stock_data.iloc[-1]
        latest_timestamp = latest_row.name[1] if isinstance(latest_row.name, tuple) else latest_row.name
        if not isinstance(latest_timestamp, pd.Timestamp):
            try:
                latest_timestamp = pd.to_datetime(latest_timestamp)
            except ValueError:
                print(f"Warning: Could not convert latest_timestamp ({latest_timestamp}) to datetime for regime check.")
                latest_timestamp = None
    except IndexError:
         print(f"Warning: Cannot determine regime for {symbol}, could not get latest row.")
         return "UNKNOWN"
    except Exception as e:
         print(f"Warning: Error extracting latest row/timestamp for regime check: {e}")
         return "UNKNOWN"

    # *** Use BBB value and threshold ***
    last_bbb = latest_row.get(bbb_col, np.nan)
    threshold = latest_row.get(bbb_threshold_col, np.nan)
    symbol = latest_row.name[0] if isinstance(latest_row.name, tuple) and 'symbol' in stock_data.index.names else 'stock'
    date_str = latest_timestamp.date() if latest_timestamp else "Unknown Date"

    if pd.isna(last_bbb) or pd.isna(threshold):
        if pd.isna(threshold):
             print(f"Warning: Cannot determine regime for {symbol} on {date_str}, BBB threshold is NaN (Insufficient history or calculation issue?).")
        elif pd.isna(last_bbb):
             print(f"Warning: Cannot determine regime for {symbol} on {date_str}, Last BBB is NaN.")
        else:
             print(f"Warning: Cannot determine regime for {symbol} on {date_str}, NaN values in latest BBB or threshold.")
        return "UNKNOWN"

    # Regime determination logic based on BBB vs threshold
    # Wider bands (higher BBB %) suggest trending
    if last_bbb > threshold:
        return "TRENDING"
    else:
        return "RANGE_BOUND"


def get_portfolio_details():
    """Gets current account equity and positions."""
    try:
        account = trading_client.get_account()
        equity = float(account.equity)
        positions = trading_client.get_all_positions()
        position_map = {pos.symbol: float(pos.qty) for pos in positions}
        return equity, position_map
    except Exception as e:
        print(f"Error getting portfolio details: {e}")
        return 0, {}

def place_market_order(symbol, qty, side):
    """Places a market order using TradingClient."""
    order_qty = int(qty)
    if order_qty <= 0:
        print(f"Warning: Attempted to place order for non-positive integer quantity ({order_qty}) of {symbol}. Original float qty: {qty:.4f}. Skipping.")
        return None
    try:
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=order_qty,
            side=side,
            time_in_force=TimeInForce.DAY
        )
        order = trading_client.submit_order(order_data=market_order_data)
        print(f"Placed {side.value} order for {order_qty} shares of {symbol}. Order ID: {order.id}")
        return order
    except Exception as e:
        print(f"Error placing {side.value} order for {symbol}: {e}")
        return None

# --- Rebalancing Logic ---

def rebalance_portfolio():
    """Main function to run the rebalancing logic daily."""
    print(f"\n--- Starting Rebalance Cycle: {datetime.now()} ---")

    equity, current_positions = get_portfolio_details()
    if equity <= 0:
        print("Error: Could not retrieve valid equity or equity is zero/negative. Skipping rebalance.")
        return

    print(f"Current Equity: ${equity:.2f}")
    print(f"Current Positions: {current_positions}")

    # --- Define date range for historical data ---
    now_utc = datetime.now(tz=timezone.utc)
    end_date_dt = now_utc.replace(hour=0, minute=0, second=0, microsecond=0)
    start_date_dt = end_date_dt - timedelta(days=LOOKBACK_DAYS_FOR_BBB + SMA_LONG + 30) # Adjusted lookback name

    # Fetch data for all stocks at once
    print("Fetching historical data...")
    all_data = get_historical_data(STOCKS, TimeFrame.Day, start_date_dt, end_date_dt)

    # Check if data fetching was successful and returned expected columns
    if all_data.empty or not isinstance(all_data.index, pd.MultiIndex) or 'symbol' not in all_data.index.names or 'close' not in all_data.columns:
        print("Error: Failed to fetch valid historical data or data is not MultiIndex with 'symbol' level. Skipping rebalance.")
        return

    # Calculate indicators for the entire DataFrame
    print("Calculating indicators...")
    all_data_indicators = calculate_indicators(all_data)
    if all_data_indicators.empty:
         print("Error: Calculation of indicators resulted in empty DataFrame (check indicator logic). Skipping rebalance.")
         return
    print("Indicators calculated.")

    # --- Cancel existing non-filled orders ---
    try:
        print("Canceling open orders...")
        cancel_statuses = trading_client.cancel_orders()
        if cancel_statuses:
             print(f"Cancel order results:")
             for status in cancel_statuses:
                 try:
                      print(f"  Order ID: {status.id}, Status: {status.status}")
                 except AttributeError:
                      print(f"  Could not parse cancel status object: {status}")
        else:
             print("No open orders to cancel.")
        time.sleep(2) # Give time for cancellations to process
    except Exception as e:
        print(f"Could not cancel all orders: {e}")


    # --- Iterate through each stock ---
    # Group by symbol level of the index to process each stock
    symbol_level_name = 'symbol' # Assuming the level name is 'symbol'
    symbol_level_index = all_data_indicators.index.names.index(symbol_level_name)
    for symbol, stock_data_indicators in all_data_indicators.groupby(level=symbol_level_index):
        print(f"\nProcessing {symbol}...")

        if stock_data_indicators.empty or stock_data_indicators.index.empty:
            print(f"No indicator data available for {symbol} after grouping. Skipping.")
            continue

        # Get the latest data row safely (this will be previous day's data)
        try:
            latest = stock_data_indicators.iloc[-1]
            # Extract timestamp from MultiIndex tuple
            latest_index_tuple = stock_data_indicators.index[-1]
            if isinstance(latest_index_tuple, tuple) and len(latest_index_tuple) > 1:
                 latest_timestamp = latest_index_tuple[1] # Get the timestamp element
            else:
                 latest_timestamp = latest_index_tuple
                 print(f"Warning: Unexpected index format for {symbol}: {latest_index_tuple}")

            # Ensure it's a datetime object before calling .date()
            if not isinstance(latest_timestamp, pd.Timestamp):
                 try:
                      latest_timestamp = pd.to_datetime(latest_timestamp)
                 except ValueError:
                      print(f"Warning: Could not convert latest_timestamp ({latest_timestamp}) to datetime for {symbol}.")
                      latest_timestamp = None
        except IndexError:
            print(f"Error accessing latest indicator data for {symbol}. Skipping.")
            continue
        except Exception as e: # Catch other potential errors during timestamp extraction
             print(f"Error processing latest index/timestamp for {symbol}: {e}")
             continue


        # Use the close price from the latest available data (previous day)
        current_price = latest.get('close', np.nan) # Use .get for safety
        if pd.isna(current_price) or current_price <= 0:
             date_str = latest_timestamp.date() if latest_timestamp else "Unknown Date"
             print(f"Warning: Invalid closing price ({current_price}) found for {symbol} on {date_str}. Skipping.")
             continue

        # Format date string safely
        date_str = latest_timestamp.date() if latest_timestamp else "Unknown Date"
        print(f"Using Previous Close Price: {current_price:.2f} from {date_str}")

        # Determine Market Regime using the indicator data for this specific stock
        regime = determine_regime(stock_data_indicators) # Pass the grouped data
        print(f"Market Regime: {regime}")

        # Get current position quantity for this stock
        current_qty = float(current_positions.get(symbol, 0.0))
        target_qty_float = 0.0 # Use float for calculations

        # Define indicator values from the latest data safely using .get()
        bbl_col = f'BBL_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        bbm_col = f'BBM_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        bbu_col = f'BBU_{BBANDS_PERIOD}_{float(BBANDS_STD_DEV)}'
        rsi_col = f'RSI_{RSI_PERIOD}'
        roc_col = f'ROC_{ROC_PERIOD}'

        lower_bb = latest.get(bbl_col, np.nan)
        middle_bb = latest.get(bbm_col, np.nan)
        upper_bb = latest.get(bbu_col, np.nan)
        rsi = latest.get(rsi_col, np.nan)
        roc = latest.get(roc_col, np.nan)

        # Check if essential indicators for strategy logic are available in the latest row
        # Regime check uses BBB and threshold, strategy logic uses BBL/BBM/BBU/RSI/ROC
        essential_strategy_cols = [lower_bb, middle_bb, upper_bb, rsi, roc]
        if any(pd.isna(val) for val in essential_strategy_cols):
            print(f"Warning: Missing essential indicator values for strategy logic for {symbol} on {date_str}. Skipping position adjustment.")
            print(f"  Values - LowerBB: {lower_bb}, MiddleBB: {middle_bb}, UpperBB: {upper_bb}, RSI: {rsi}, ROC: {roc}")
            target_qty_float = current_qty # Hold position
            target_qty_int = int(target_qty_float)
        else:
            # --- Apply Strategy Based on Regime ---
            # Regime check already happened, proceed with logic
            if regime == "RANGE_BOUND":
                print("Applying Mean Reversion Strategy...")
                if current_qty > 0 and (current_price > middle_bb or rsi > 50):
                    print(f"[Prev Day Data] Exit Long Signal: Price ({current_price:.2f}) > Middle BB ({middle_bb:.2f}) or RSI ({rsi:.2f}) > 50. Target Flat.")
                    target_qty_float = 0.0
                elif current_qty == 0 and current_price < lower_bb and rsi < RSI_OVERSOLD:
                    print(f"[Prev Day Data] Entry Long Signal: Price ({current_price:.2f}) < Lower BB ({lower_bb:.2f}) and RSI ({rsi:.2f}) < {RSI_OVERSOLD}.")
                    target_capital = equity * CAPITAL_PER_STOCK_PERCENT
                    target_qty_float = target_capital / current_price
                    print(f"Target Buy Qty (float): {target_qty_float:.4f}")
                else:
                    target_qty_float = current_qty
                    print("[Prev Day Data] Hold Signal: No entry/exit criteria met.")

            elif regime == "TRENDING":
                print("Applying Momentum Strategy...")
                if current_qty > 0 and (current_price < middle_bb or roc < 0):
                     print(f"[Prev Day Data] Exit Long Signal: Price ({current_price:.2f}) < Middle BB ({middle_bb:.2f}) or ROC ({roc:.2f}) < 0. Target Flat.")
                     target_qty_float = 0.0
                elif current_qty == 0 and current_price > upper_bb and roc > 0:
                     print(f"[Prev Day Data] Entry Long Signal: Price ({current_price:.2f}) > Upper BB ({upper_bb:.2f}) and ROC ({roc:.2f}) > 0.")
                     target_capital = equity * CAPITAL_PER_STOCK_PERCENT
                     target_qty_float = target_capital / current_price
                     print(f"Target Buy Qty (float): {target_qty_float:.4f}")
                else:
                     target_qty_float = current_qty
                     print("[Prev Day Data] Hold Signal: No entry/exit criteria met.")

            elif regime == "UNKNOWN":
                # This regime is determined if BBB or threshold is NaN in the latest row
                print("Regime Unknown (likely due to NaN indicators for regime). Holding position.")
                target_qty_float = current_qty

            target_qty_int = int(target_qty_float)


        # --- Adjust Position ---
        qty_diff = target_qty_int - int(current_qty)
        print(f"Current Qty: {current_qty:.4f}, Target Qty (Int): {target_qty_int}, Diff: {qty_diff}")

        if qty_diff > 0:
            print(f"Action: Buy {qty_diff} shares of {symbol}")
            place_market_order(symbol, qty_diff, OrderSide.BUY)
        elif qty_diff < 0:
            sell_qty = min(abs(qty_diff), int(current_qty))
            if sell_qty > 0:
                print(f"Action: Sell {sell_qty} shares of {symbol}")
                place_market_order(symbol, sell_qty, OrderSide.SELL)
            else:
                print(f"Action: Hold - Calculated sell quantity is zero or negative (Current: {current_qty:.4f}, Target: {target_qty_int}).")
        else:
            print(f"Action: Hold - No change needed for {symbol}")

        time.sleep(1) # Small delay between processing stocks

    print(f"\n--- Rebalance Cycle Complete: {datetime.now()} ---")


# --- Main Execution Loop ---
if __name__ == "__main__":
    # You would typically run this function on a schedule (e.g., daily cron job)
    # For demonstration, we run it once.
    rebalance_portfolio()

    # Example of how you might run it periodically (conceptual)
    # import schedule # Requires 'pip install schedule'
    # schedule.every().day.at("15:45").do(rebalance_portfolio) # Example: Run near market close EST
    # print("Scheduler started. Waiting for scheduled job...")
    # while True:
    #     schedule.run_pending()
    #     time.sleep(60) # Check every minute



--- Starting Rebalance Cycle: 2025-06-25 13:56:58.767711 ---
Current Equity: $118826.05
Current Positions: {'JNJ': 711.0}
Fetching historical data...
Requesting data from 2025-03-24 00:00:00+00:00 to 2025-06-25 00:00:00+00:00
Calculating indicators...
Calculating indicators group-wise for MultiIndex DataFrame...
BBands columns added. Columns now: ['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap', 'SMA_10', 'SMA_30', 'RSI_10', 'ROC_10', 'BBL_20_2.0', 'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0']
Indicator calculation complete. DataFrame shape: (1344, 17)
Indicators calculated.
Canceling open orders...
No open orders to cancel.

Processing AAPL...
Using Previous Close Price: 200.30 from 2025-06-24
Market Regime: RANGE_BOUND
Applying Mean Reversion Strategy...
[Prev Day Data] Hold Signal: No entry/exit criteria met.
Current Qty: 0.0000, Target Qty (Int): 0, Diff: 0
Action: Hold - No change needed for AAPL

Processing ABBV...
Using Previous Close Price: 185.55 fro